# Step 1: Import Libraries
Here we are going to import our necessary libraries. 

In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/dataset4/nfl_season_data (4).csv
/kaggle/input/mergedffdata/nfl_season_data.csv


# Check the NFL_Data_Py Index
* Here we used the NFL Data Py index to start collecting our data from seasons dating back to 2020. 
* We want to make sure we make the right calls to collect the necessary data. 

In [5]:
import nfl_data_py as nfl

# Load weekly data for the years you want
weekly_data = nfl.import_weekly_data([2020, 2021, 2022, 2023])

# Display all available columns to identify the correct names
print(weekly_data.columns)


Downcasting floats.
Index(['player_id', 'player_name', 'player_display_name', 'position',
       'position_group', 'headshot_url', 'recent_team', 'season', 'week',
       'season_type', 'opponent_team', 'completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr', 'special_teams_t

# Using NFL_Data_Py For Fantasy Prediction Dataset. 
Here we began merging our data into on set and verifying all of our columns merged successfully. We also want to go through and check for duplicates. 

In [ ]:
# Verify your columns are present and named correctly. 
print(players.head())
print(players.columns)


In [ ]:
# Strip any leading or trailing spaces from column names
players.columns = players.columns.str.strip()

# Now try the merge again
merged_fantasy = pd.merge(
    fantasy_data, 
    players[['player_id', 'player_name', 'position', 'team']], 
    on='player_id', 
    how='left'
)


In [ ]:
print(players['player_id'].dtype)
print(fantasy_data['player_id'].dtype)


In [ ]:
# Perform a basic merge with just player_id and team
test_merge = pd.merge(
    fantasy_data, 
    players[['player_id', 'team']], 
    on='player_id', 
    how='left'
)

print(test_merge.head())


In [ ]:
# Strip any spaces again, just to be certain
players.columns = players.columns.str.strip()

# Display the columns to verify
print(players.columns)


In [ ]:
print(players[['player_id', 'player_name', 'position', 'team']].head(10))


In [ ]:
# Test merge with only the 'player_id' and 'team' columns
test_merge = pd.merge(
    fantasy_data, 
    players[['player_id', 'team']], 
    on='player_id', 
    how='left'
)

print(test_merge.head())


In [ ]:
# Check if there are any weird characters or encoding issues
print(players['team'].unique())


In [ ]:
import nfl_data_py as nfl
import pandas as pd

# Specify the relevant columns you want to include
columns = [
    'player_id', 'player_name', 'position', 'season', 'week', 'opponent_team',
    'completions', 'attempts', 'passing_yards', 'passing_tds', 'interceptions', 'carries', 
    'rushing_yards', 'rushing_tds', 'receptions', 'targets', 'receiving_yards', 'receiving_tds', 
    'target_share', 'air_yards_share', 'wopr', 'racr', 'fantasy_points', 'fantasy_points_ppr'
]

# Load the weekly data with the relevant columns
fantasy_data = nfl.import_weekly_data([2020, 2021, 2022, 2023], columns=columns)

# Load player metadata for merging
players = nfl.import_seasonal_rosters([2020, 2021, 2022, 2023])

# Rename the 'team' column in the players dataset to avoid conflicts
players.rename(columns={'team': 'player_team'}, inplace=True)

# Perform the full merge with renamed columns
merged_fantasy = pd.merge(
    fantasy_data, 
    players[['player_id', 'player_name', 'position', 'player_team']], 
    on='player_id', 
    how='left'
)

# Preview the final merged data
print(merged_fantasy.head())


# Advanced Statistics

In [ ]:
import pandas as pd

# Filter for the last 3 seasons and relevant positions
positions_to_include = ['QB', 'RB', 'WR', 'TE',''K']
filtered_data = cleaned_data[
    (cleaned_data['season'].isin([2021, 2022, 2023])) &
    (cleaned_data['position_y'].isin(positions_to_include))
]

# Group by player and position, then aggregate their total fantasy points across seasons
aggregated_data = filtered_data.groupby(['player_name_y', 'position_y', 'player_team'], as_index=False).agg({
    'fantasy_points_ppr': 'sum'
})

# Get the top 10 players for each relevant position
top_players_by_position = (
    aggregated_data.groupby('position_y', group_keys=False)
    .apply(lambda x: x.nlargest(10, 'fantasy_points_ppr'))
    .reset_index(drop=True)
)

# Display all rows in the output (adjust as needed)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Display the top players for each position
print(top_players_by_position[['player_name_y', 'position_y', 'fantasy_points_ppr', 'player_team']])


# To gather recent injury data: 
In the NFL and Fantasy, injuries play a huge part in a teams success or failure. We want to be able to have our model predict the likelihood of future injuries based of recent and past injury history. This one may be a bit tricky but it will be fun to see how the precitions accuracy plays out. TBD....

In [ ]:
import nfl_data_py as nfl
import pandas as pd

# Import injury data for the last 3 years
injury_data = nfl.import_injuries([2021, 2022, 2023])

# Preview the injury data
print(injury_data.head())

# Standardize the naming conventions in both datasets for better matching
injury_data['full_name'] = injury_data['first_name'] + ' ' + injury_data['last_name']

# Merge the injury data into your existing fantasy football dataset
merged_data = pd.merge(
    cleaned_data, 
    injury_data[['full_name', 'season', 'team', 'report_primary_injury', 'report_status', 'practice_status']],  # Adjust columns as needed
    left_on=['player_name_y', 'player_team', 'season'],
    right_on=['full_name', 'team', 'season'],
    how='left'
)

# Preview the updated dataset
print(merged_data.head())


# Steps for Cleaning Up the Merged Data:
Remove Unnecessary Columns: You now have columns like player_name_x and player_name_y, which are duplicates. You can clean this up:

In [ ]:
import nfl_data_py as nfl

# Clean your merged dataset using the clean_nfl_data function
cleaned_data = nfl.clean_nfl_data(merged_data)  # Note that we're assigning the output to 'cleaned_data'

# Now 'cleaned_data' should be standardized
print(cleaned_data.head())




In [ ]:
print(cleaned_data.tail(10))  # Shows the first 10 rows


In [ ]:
# Save the cleaned dataset to a CSV file
cleaned_data.to_csv('nfl_season_data.csv', index=False)

print('nfl_season_data.csv')


In [ ]:
import nfl_data_py as nfl
import pandas as pd

# Import weekly data for the past 3 seasons
weekly_data = nfl.import_weekly_data([2021, 2022, 2023])

# Preview the weekly data
print(weekly_data.head())

# Merge weekly data with your existing fantasy dataset
merged_weekly_data = pd.merge(
    merged_data,  # Your existing merged dataset with injury data
    weekly_data[['player_id', 'season', 'week', 'team', 'opponent', 'game_location', 'stadium', 'fantasy_points_ppr']],  # Adjust columns as needed
    on=['player_id', 'season', 'week', 'team'],
    how='left'
)

# Preview the updated dataset
print(merged_weekly_data.head())


In [ ]:
import nfl_data_py as nfl
import pandas as pd
# Import the schedule for the upcoming season
upcoming_schedule = nfl.import_schedules([2024])  # Adjust the year to the upcoming season

# Preview the schedule data
print(upcoming_schedule.head())

# Upcoming Projections and Matchups
Once our model is trained and makes accurate predicitions on our 2023 datasets we can introduce upcoming games to the model and see how it does. 2024 will be a year long test for it but we will tweak and tune so that we get the accuracy we are hoping for. 

In [ ]:
import nfl_data_py as nfl
import pandas as pd

# Import the schedule for the upcoming season
upcoming_schedule = nfl.import_schedules([2024])  # Adjust the year to the upcoming season

# Preview the schedule data
print(upcoming_schedule.head())

# Load your cleaned data from a CSV file
cleaned_data = pd.read_csv('/kaggle/input/mergedffdata/nfl_season_data.csv')

# Merge the upcoming schedule with your existing fantasy data
upcoming_data = pd.merge(
    cleaned_data,  # Your existing dataset
    upcoming_schedule[['week', 'home_team', 'away_team', 'gametime', 'stadium', 'weekday']],  # Corrected column names
    left_on=['player_team'],  # Adjust if needed
    right_on=['home_team'],  # Adjust if needed
    how='left'
)

# Preview the updated dataset
print(upcoming_data.head())


In [ ]:
# Save the updated dataset to the CSV file
upcoming_data.to_csv('/kaggle/working/nfl_season_data.csv', index=False)

In [ ]:
import pandas as pd
import os

# Load the dataset
file_path = '/kaggle/working/nfl_season_data.csv'
large_data = pd.read_csv(file_path)

# Calculate approximate chunk size based on desired file size (e.g., 100 MB)
desired_file_size_mb = 100  # Adjust as needed
row_size_in_bytes = large_data.memory_usage(deep=True).sum() / len(large_data)
rows_per_chunk = int((desired_file_size_mb * 1024 * 1024) / row_size_in_bytes)

# Split the dataset into chunks
for i, chunk in enumerate(range(0, large_data.shape[0], rows_per_chunk)):
    chunk_data = large_data.iloc[chunk:chunk + rows_per_chunk]
    chunk_data.to_csv(f'/kaggle/working/nfl_season_data_part_{i}.csv', index=False)

# Check saved file sizes
for filename in os.listdir('/kaggle/working/'):
    if filename.startswith('nfl_season_data_part_'):
        print(f"{filename}: {os.path.getsize(f'/kaggle/working/{filename}') / (1024 * 1024):.2f} MB")


In [ ]:
import os

# Check saved file sizes
for filename in os.listdir('/kaggle/working/'):
    if filename.startswith('nfl_season_data_part_0'):
        print(f"{filename}: {os.path.getsize(f'/kaggle/working/{filename}') / (1024 * 1024):.2f} MB")


In [ ]:
import pandas as pd
import nfl_data_py as nfl
import os

# Directory where the split files are located
directory = '/kaggle/working/'

# Loop through and clean each file
for filename in os.listdir(directory):
    if filename.startswith('nfl_season_data_part_'):
        filepath = os.path.join(directory, filename)
        
        # Load the file
        data = pd.read_csv(filepath)
        
        # Run the clean_nfl_data command
        cleaned_data = nfl.clean_nfl_data(data)
        
        # Save the cleaned file
        cleaned_filepath = filepath.replace('.csv', '_cleaned.csv')
        cleaned_data.to_csv(cleaned_filepath, index=False)

        print(f"Cleaned and saved: {cleaned_filepath}")

In [ ]:
import pandas as pd

# List of your split files
files = [f'/kaggle/working/nfl_season_data_part_{i}.csv' for i in range(17)]

# Loop through each file, clean, and save with new names
for i, file in enumerate(files):
    try:
        # Read each CSV file with error handling
        df = pd.read_csv(file, on_bad_lines='skip', low_memory=False)
        
        # Save the cleaned file with a new name
        cleaned_file_name = f'/kaggle/working/cleaned_file_{i}.csv'
        df.to_csv(cleaned_file_name, index=False)
        
        print(f'Successfully saved: {cleaned_file_name}')
    
    except Exception as e:
        print(f"Error processing file {file}: {str(e)}")



# NFL Def Statistics
Here we will begin to collect our NFL Def stats because as you know, Defense wins championships and no team is complete without a trustworth Def holding down their team. 

In [ ]:
# Import the defensive data for a given season
team_defense_data = nfl.import_defense_data([2023])  # You can change the year as needed

# Save the defensive data to a CSV file
team_defense_data.to_csv('nfl_team_defense_data.csv', index=False)

# Display the first few rows
print(team_defense_data.head())


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = 'https://www.pro-football-reference.com/years/2023/opp.htm'

all_teams_data = []

# Loop through all teams for the seasons you need
for year in range(2020, 2024):
    for week in range(1, 18):  # 17 weeks in each season
        url = base_url.format(year, week)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Example of extracting table data
        table = soup.find('table', {'id': 'team_stats'})
        if table:
            df = pd.read_html(str(table))[0]
            df['Year'] = year
            df['Week'] = week
            all_weeks_data.append(df)

# Combine all the weeks into one DataFrame
combined_data = pd.concat(all_weeks_data, ignore_index=True)
combined_data.to_csv('nfl_weekly_data.csv', index=False)


# Reload command

In [ ]:
!pip install nfl_data_py


In [ ]:
import nfl_data_py as nfl
import pandas as pd

# Reload your merged dataset
merged_data = pd.read_csv('/kaggle/input/mergedffdata/nfl_season_data.csv')


# You might also need to reload any merged datasets or functions you’ve previously defined


In [ ]:
import pandas as pd

# Load the initial file
df = pd.read_csv('/kaggle/working/nfl_season_data.csv')


In [9]:
import nfl_data_py as nfl
import pandas as pd

# Specify the years for which you want to import draft picks data
years = [2024]  # Replace with the desired years

# Fetch the draft picks data for the specified years
draft_picks_data = nfl.import_draft_picks(years)

# Convert to DataFrame (if it's not already)
draft_picks_df = pd.DataFrame(draft_picks_data)

# Save the DataFrame to a CSV file
draft_picks_df.to_csv('nfl_draft_picks.csv', index=False)

print("NFL draft picks data saved successfully as 'nfl_draft_picks.csv'")


NFL draft picks data saved successfully as 'nfl_draft_picks.csv'
